### NEXT STEPS:



- Quitar los *outlires*
- Probar con los "sin estandardizar"
- Mirar la importancia de cada columna para quitarlas o menos

### Sprint 2 - Random Forest

In [1]:
# Tratamiento de datos
# ------------------------------------------------------------------------------
import numpy as np
import pandas as pd

# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Modelado y evaluación
# ------------------------------------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV

# Barra de progreso de un proceso
# ------------------------------------------------------------------------------
from tqdm import tqdm

# Configuración warnings
# ------------------------------------------------------------------------------
import warnings
warnings.filterwarnings('once')

In [3]:
df = pd.read_csv("../datos/bikes_casual_estan.csv", index_col = 0)
df.head()

,instant,yr,holiday,workingday,atemp,hum,windspeed,casual,weekday,weathersit_map,season_map
0,1,0,1,0,-0.456399,0.855582,-0.208909,331,1,1,0
1,2,0,0,1,-0.492863,0.332722,0.687550,131,4,1,0
2,3,0,0,1,-1.098990,-0.900337,0.685210,120,4,2,0
3,4,0,0,0,-1.015201,-0.170632,-0.210435,108,7,2,1
4,5,0,0,1,-0.951952,-0.901843,0.060294,82,2,2,1


In [6]:
# al igual que en la regresión lineal tenemos que separar nuestro dataframe en X e y

X = df.drop("casual", axis = 1)
y = df["casual"]

In [7]:
# y dividir nuestros datos en train y test para poder evaluar la bondad de nuestro modelo

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [36]:
# recordamos que lo primero que hacíamos era crear un diccionario con los hiperparámetros que queríamos probar

param = {"max_depth": [22], # teniendo en cuenta que teníamos overfitting tendremos que reducir la profundidad del modelo, la nuestra anterior era de 17. Bajaremos mucho este valor ya que teníamos un overfitting muy claro
        "max_features": [4],# calculamos en celdas anteriores, probaremos a hacer el modelo como una variable, 2, 3 y 4. Ponemos como límite el 4 ya que es el resultado de la raiz cuadrada. 
        # estos dos hiperparámetros son más difíciles de definir, pero usualmente se suelen elegir los siguientes valores
        "min_samples_split":[8],
        "min_samples_leaf": [8]} 

In [37]:
gs_rf = GridSearchCV(
            estimator=RandomForestRegressor(), # tipo de modelo que queremos hacer
            param_grid= param, # que hiperparámetros queremos que testee
            cv=10, # crossvalidation que aprendimos en la lección de regresión lineal intro. 
            verbose=-1, # para que no nos printee ningún mensaje en pantalla
            return_train_score = True, # para que nos devuelva el valor de las métricas de set de datos de entrenamiento
            scoring="neg_mean_squared_error") # la métrica que queremos que nos devuelva

In [38]:
# ajustamos el modelo que acabamos de definir en el GridSearch
# 🚨 Esta celda puede tardar en ejecutarse

gs_rf.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [22], 'max_features': [4],
                         'min_samples_leaf': [8], 'min_samples_split': [8]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [39]:
# al igual que el decision tree podemos sacar cual es nuestro mejor bosque
# En este caso,  nuestro mejor bosque es aquel que esta formado por arboles de 6 de profundidad, usa 4 variables y que tiene  un min_samples_leaf y un min_samples_split de 10. 

bosque = gs_rf.best_estimator_
bosque

RandomForestRegressor(max_depth=22, max_features=4, min_samples_leaf=8,
                      min_samples_split=8)

In [40]:
y_pred_test_rf = bosque.predict(x_test)
y_pred_train_rf = bosque.predict(x_train)

In [41]:
def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    
    
    resultados = {'MAE': [mean_absolute_error(y_test, y_test_pred), mean_absolute_error(y_train, y_train_pred)],
                'MSE': [mean_squared_error(y_test, y_test_pred), mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_test_pred)), np.sqrt(mean_squared_error(y_train, y_train_pred))],
                'R2':  [r2_score(y_test, y_test_pred), r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

In [42]:
# sacamos las métricas para ver si hay overfitting o unerfitting, para modificar la profundidad en función de estos resultados

dt_results = metricas(y_test, y_train, y_pred_test_rf, y_pred_train_rf, "Random Forest")
dt_results

,MAE,MSE,RMSE,R2,set,modelo
0,295.790537,182846.141842,427.605124,0.619407,test,Random Forest
1,277.227152,155847.529682,394.775290,0.666783,train,Random Forest


In [44]:
df_r = pd.read_csv("../datos/bikes_registered_estan.csv", index_col = 0)
df_r.head()

,instant,yr,holiday,workingday,atemp,hum,windspeed,registered,weekday,weathersit_map,season_map
0,1,0,1,0,-0.456399,0.855582,-0.208909,654,1,1,0
1,2,0,0,1,-0.492863,0.332722,0.687550,670,4,1,0
2,3,0,0,1,-1.098990,-0.900337,0.685210,1229,4,2,0
3,4,0,0,0,-1.015201,-0.170632,-0.210435,1454,7,2,1
4,5,0,0,1,-0.951952,-0.901843,0.060294,1518,2,2,1


In [47]:
# al igual que en la regresión lineal tenemos que separar nuestro dataframe en X e y

X2 = df_r.drop("registered", axis = 1)
y2 = df_r["registered"]

In [49]:
# y dividir nuestros datos en train y test para poder evaluar la bondad de nuestro modelo

x2_train, x2_test, y2_train, y2_test = train_test_split(X2, y2, test_size = 0.2, random_state = 42)

In [106]:
# recordamos que lo primero que hacíamos era crear un diccionario con los hiperparámetros que queríamos probar

param2 = {"max_depth": [16], # teniendo en cuenta que teníamos overfitting tendremos que reducir la profundidad del modelo, la nuestra anterior era de 17. Bajaremos mucho este valor ya que teníamos un overfitting muy claro
        "max_features": [4],# calculamos en celdas anteriores, probaremos a hacer el modelo como una variable, 2, 3 y 4. Ponemos como límite el 4 ya que es el resultado de la raiz cuadrada. 
        # estos dos hiperparámetros son más difíciles de definir, pero usualmente se suelen elegir los siguientes valores
        "min_samples_split":[8],
        "min_samples_leaf": [8]} 

In [107]:
gs_rf = GridSearchCV(
            estimator=RandomForestRegressor(), # tipo de modelo que queremos hacer
            param_grid= param2, # que hiperparámetros queremos que testee
            cv=10, # crossvalidation que aprendimos en la lección de regresión lineal intro. 
            verbose=-1, # para que no nos printee ningún mensaje en pantalla
            return_train_score = True, # para que nos devuelva el valor de las métricas de set de datos de entrenamiento
            scoring="neg_mean_squared_error") # la métrica que queremos que nos devuelva

In [108]:
# ajustamos el modelo que acabamos de definir en el GridSearch
# 🚨 Esta celda puede tardar en ejecutarse

gs_rf.fit(x2_train, y2_train)

GridSearchCV(cv=10, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [16], 'max_features': [4],
                         'min_samples_leaf': [8], 'min_samples_split': [8]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [109]:
# al igual que el decision tree podemos sacar cual es nuestro mejor bosque
# En este caso,  nuestro mejor bosque es aquel que esta formado por arboles de 6 de profundidad, usa 4 variables y que tiene  un min_samples_leaf y un min_samples_split de 10. 

bosque = gs_rf.best_estimator_
bosque

RandomForestRegressor(max_depth=16, max_features=4, min_samples_leaf=8,
                      min_samples_split=8)

In [110]:
y_pred_test_rf = bosque.predict(x_test)
y_pred_train_rf = bosque.predict(x_train)

In [111]:
def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    
    
    resultados = {'MAE': [mean_absolute_error(y_test, y_test_pred), mean_absolute_error(y_train, y_train_pred)],
                'MSE': [mean_squared_error(y_test, y_test_pred), mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_test_pred)), np.sqrt(mean_squared_error(y_train, y_train_pred))],
                'R2':  [r2_score(y_test, y_test_pred), r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

In [112]:
# sacamos las métricas para ver si hay overfitting o unerfitting, para modificar la profundidad en función de estos resultados

dt_results = metricas(y_test, y_train, y_pred_test_rf, y_pred_train_rf, "Random Forest")
dt_results

,MAE,MSE,RMSE,R2,set,modelo
0,509.387673,422195.261729,649.765544,0.804410,test,Random Forest
1,483.600776,411937.971127,641.823941,0.834768,train,Random Forest


Nos quedamos con *registered* porque tiene el R2 mejor.

- Quitar los put